In [1]:
import datetime
import os, shutil
import time
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import image, ImageDataGenerator, img_to_array ,array_to_img, load_img
np.random.seed(123)

Using TensorFlow backend.


In [2]:
# dimensions of our images.  
img_width, img_height = 224, 224  
   
train_data_dir = '../data/train'  
validation_data_dir = '../data/validation'  
test_data_dir = '../data/test'
   
# number of epochs to train top model  
epochs = 100  
# batch size used by flow_from_directory and predict_generator  
batch_size = 50

In [3]:
datagen = ImageDataGenerator(rescale=1./255) 

In [4]:
from keras.applications import VGG19

In [5]:
cnn_base = VGG19(weights='imagenet',
                 include_top=False,
                 input_shape=(64,64,3)
                )

In [6]:
cnn_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [17]:
def extract_features(directory, sample_amount):
    features = np.zeros(shape=(sample_amount, 2, 2, 512)) 
    labels = np.zeros(shape=(sample_amount))
    generator = datagen.flow_from_directory(
        directory, 
        target_size=(64, 64), 
        batch_size = batch_size, 
        class_mode='binary',
        shuffle=False)
    i=0

    for inputs_batch, labels_batch in generator:
        features_batch = cnn_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch 
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i = i + 1
        if i * batch_size >= sample_amount:
            break
    return features, labels

In [18]:
# you should be able to divide sample_amount by batch_size!!
train_features, train_labels = extract_features(train_data_dir, 100) # 13412
validation_features, validation_labels = extract_features(validation_data_dir, 100) #2549 
test_features, test_labels = extract_features(test_data_dir, 100) #1845

train_features = np.reshape(train_features, (100, 2 * 2 * 512))
validation_features = np.reshape(validation_features, (100, 2 * 2 * 512))
test_features = np.reshape(test_features, (100, 2 * 2 * 512))

Found 13412 images belonging to 6 classes.
Found 2549 images belonging to 6 classes.
Found 1845 images belonging to 6 classes.


In [19]:
test_features.shape

(100, 2048)

In [20]:
train_features.shape

(100, 2048)

In [21]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()

model.add(layers.Dense(256, 
                       activation='relu', 
                       input_dim=2 * 2 * 512))

model.add(layers.Dense(6, 
                       activation='sigmoid'))


In [22]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, 
                    train_labels,
                    epochs=20,
                    batch_size=10,
                    validation_data=(validation_features, 
                                     validation_labels))

ValueError: Error when checking target: expected dense_2 to have shape (6,) but got array with shape (1,)

# Image functions

In [ ]:
import matplotlib.image as mpimg

In [ ]:
def read_image(file_path, resize=True):
    image_path = file_path

    orig = mpimg.imread(image_path)  

    print("[INFO] loading and preprocessing image...")  
    image = load_img(image_path, target_size=(224, 224))  
    image = img_to_array(image)  

    # important! otherwise the predictions will be '0'
    if resize:
        image = image / 255  

    image = np.expand_dims(image, axis=0)
    return image

In [ ]:
path = '../data/test/elephants/OIP-VC6pX0aP4-OEoBF2nO997wHaIS.jpeg'

In [ ]:
def show_img(path):
    img_path = path
    img = image.load_img(img_path, target_size=(224, 224))
    print(type(img))

    x = image.img_to_array(img)
    print(type(x))
    print(x.shape)
    plt.imshow(x/255.)